In [1]:
import clip
import torch

from tqdm.notebook import tqdm

from datasets import *

In [2]:
# Global Variables
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device)

In [3]:
# Configs
phrase = "This is a {} with petals."
dataset_obj = Flowers102(4, 50)
n_shots = 4

In [5]:
# Helpers for embedding generation

def get_clip_text_features(classes):
    """Given a list of classes, generates clip embeddings per class"""
    # Assumes the positions are in accordance to the label numbers
    embedding_per_class = {}
    
    global clip_model
    global phrase

    with torch.no_grad():
        for i,_class in enumerate(classes):
            _class = _class.replace("_", " ")
            text = clip.tokenize(phrase.format(_class)).cuda() 
            class_embeddings = clip_model.encode_text(
                    text
                )
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            embedding_per_class[i] = class_embeddings
    return embedding_per_class

def get_clip_image_features(data_loader):
    """Given a dataloader object, generate two torch arrays of encoded images and corresponding labels"""
    all_features = []
    all_labels = []

    global clip_model

    with torch.no_grad():
        for images, labels in tqdm(data_loader):
            features = clip_model.encode_image(images.to(device))
            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features), torch.cat(all_labels)

In [6]:
def image_and_text_embedding_1(train_features, train_labels, test_loader, classes, a, b):
    # Currently, this version takes an average of Clip-Zero shot & average image embedding
    
    # a -> importance given to the image embedding
    # b -> importance given to the text embedding
    
    # Normalization doesn't affect performance for avg lambda curves.
    train_features = train_features/train_features.norm(dim=-1, keepdim=True)

    text_clip_features = get_clip_text_features(classes)

    text_embeds = []
    image_embeds = []
    embeddings = []
    for c in range(len(classes)):
        ind = (c == train_labels).nonzero(as_tuple=True)[0]

        # replace this with linear probe
        image_embs = torch.mean(train_features[ind],dim=0)

        text_embs = text_clip_features[c].squeeze()
        text_embeds.append(text_embs)
        image_embeds.append(image_embs)
        avg_emb = (a*image_embs + b*text_embs)
        embeddings.append(avg_emb)
    
    zeroshot_weights = torch.stack(embeddings).squeeze(1)
    image_embeds = torch.stack(image_embeds).squeeze(1)
    text_embeds = torch.stack(text_embeds).squeeze(1)

    with torch.no_grad():
        top1, n = 0.0, 0.0
        for i, (images, target) in enumerate(tqdm(test_loader)):
            images = images.cuda()
            target = target.cuda()
            image_features = clip_model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            # predict
            for image_feature,tar in zip(image_features, target):
                texts_per_img = []
                imgs_per_img = []
                for c in range(len(classes)):
                    text_embs = text_clip_features[c].squeeze()
                    texts_per_img.append(text_embs)
                    imgs_per_img.append(image_feature)
                texts_per_img = torch.stack(texts_per_img)
                imgs_per_img = torch.stack(imgs_per_img)
                dot = a*(imgs_per_img @ image_embeds.T) + b*(imgs_per_img @ text_embeds.T) + 0*(texts_per_img @ text_embeds.T) + 0*(texts_per_img @ image_embeds.T)
                out = np.unravel_index(torch.argmax(dot, axis=None).cpu().numpy(), dot.shape)[1]
                if tar.cpu().numpy() == out:
                    top1 +=1
                n+=1

    return top1/n

In [7]:
train_loader, valid_loader, _, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess,num_elements_per_class=n_shots)
test_loader, _ = dataset_obj.get_test_loader(transform_fn=clip_preprocess)
classes = dataset_obj.classes

train_features_unnorm, train_labels = get_clip_image_features(train_loader)
train_features = train_features_unnorm/train_features_unnorm.norm(dim=-1, keepdim=True)


  0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
# Truth
out = image_and_text_embedding_1(train_features, train_labels, test_loader, classes, 0.75, 0.25)
print(out)

  0%|          | 0/17 [00:00<?, ?it/s]

0.8400488400488401


### Finding Optimal Lambda - Average Embedding

In [49]:
# Initialize model
import torch.nn as nn

class Lambda(nn.Module):
    def __init__(self):
        super().__init__()
        self.lam = torch.nn.Parameter(torch.tensor([0.5]).cuda())
        self.sig = nn.Sigmoid()
        self.sig_lam = self.sig(self.lam)
        
    def forward(self, image_prototype, text_prototpe, images):
        sig_lam = self.sig(self.lam)
        self.sig_lam = sig_lam
        dot = sig_lam*(images @ image_prototype.T) + (1-sig_lam)*(images @ text_prototpe.T)
        return dot
    
    def string(self):
        return f'Lambda: {self.sig(self.lam).item()}'
    
model = Lambda()
model.logit_scale = nn.Parameter(torch.ones([], device=device))
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-5)
num_epochs = 500

In [50]:
# Replace with output of the model

lam = 0.9

In [51]:
# Generate prototypes for the lambda model

text_clip_features = get_clip_text_features(classes)
text_embeds = []
image_embeds = []

for c in range(len(classes)):
    ind = (c == train_labels).nonzero(as_tuple=True)[0]
    image_embs = torch.mean(train_features[ind],dim=0)
    text_embs = text_clip_features[c].squeeze()
    text_embeds.append(text_embs)
    image_embeds.append(image_embs)

image_embeds = torch.stack(image_embeds).squeeze(1)
text_embeds = torch.stack(text_embeds).squeeze(1)

In [52]:
# Train Lambda model over TRAIN set


total_loss = 0.0
correct = 0.0
total = 0
epoch = 0

for epoch in range(num_epochs):
    for i, (images, target) in enumerate(train_loader):
        model.train()
        images = images.cuda()
        target = target.cuda()
        image_features_unnorm = clip_model.encode_image(images)
        image_features = image_features_unnorm/image_features_unnorm.norm(dim=-1, keepdim=True)

        optimizer.zero_grad()
        outputs = model(image_embeds, text_embeds, image_features)
        loss = criterion(outputs, target)
        loss.backward(retain_graph=True)
        optimizer.step()

        total_loss+=loss.item()
        total += len(target)
        correct += torch.sum(torch.argmax(outputs, axis=1) == target).item()

    if epoch%10 == 0:

        model.eval()
        acc = evaluate(valid_loader, image_embeds, text_embeds, model.sig_lam.item())
        print(acc, model.sig_lam.item())

    print(f"Training: Epoch {epoch} || Loss: {loss.item():7.3f} || {model.string()}")
    

  0%|          | 0/17 [00:00<?, ?it/s]

0.8376068376068376 0.6227090954780579
Training: Epoch 0 || Loss:   4.494 || Lambda: 0.6227242946624756
Training: Epoch 1 || Loss:   4.500 || Lambda: 0.6230031847953796
Training: Epoch 2 || Loss:   4.506 || Lambda: 0.6232813000679016
Training: Epoch 3 || Loss:   4.494 || Lambda: 0.6235789060592651
Training: Epoch 4 || Loss:   4.498 || Lambda: 0.6238514184951782
Training: Epoch 5 || Loss:   4.491 || Lambda: 0.6241243481636047
Training: Epoch 6 || Loss:   4.489 || Lambda: 0.6243962645530701
Training: Epoch 7 || Loss:   4.500 || Lambda: 0.6246746778488159
Training: Epoch 8 || Loss:   4.487 || Lambda: 0.6249479651451111
Training: Epoch 9 || Loss:   4.500 || Lambda: 0.6252186298370361


  0%|          | 0/17 [00:00<?, ?it/s]

0.8388278388278388 0.6254631280899048
Training: Epoch 10 || Loss:   4.499 || Lambda: 0.6255016922950745
Training: Epoch 11 || Loss:   4.503 || Lambda: 0.6257757544517517
Training: Epoch 12 || Loss:   4.484 || Lambda: 0.6260555982589722
Training: Epoch 13 || Loss:   4.497 || Lambda: 0.6263313293457031
Training: Epoch 14 || Loss:   4.513 || Lambda: 0.6266129612922668
Training: Epoch 15 || Loss:   4.497 || Lambda: 0.6268895268440247
Training: Epoch 16 || Loss:   4.502 || Lambda: 0.6271708011627197
Training: Epoch 17 || Loss:   4.473 || Lambda: 0.6274455785751343
Training: Epoch 18 || Loss:   4.513 || Lambda: 0.6277226209640503
Training: Epoch 19 || Loss:   4.480 || Lambda: 0.6279940605163574


  0%|          | 0/17 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff3b69330d0>
Traceback (most recent call last):
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff3b69330d0>
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
Traceback (most recent call last):
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/multiprocessing/process.py", line 147, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
    self._shutdown_workers()
AssertionError: can only join a child process
  File "/nethome/b

Training: Epoch 147 || Loss:   4.495 || Lambda: 0.6618058681488037


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff3b69330d0>
Traceback (most recent call last):
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/multiprocessing/process.py", line 147, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff3b69330d0>
Traceback (most recent call last):
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/nethome/b

Training: Epoch 148 || Loss:   4.490 || Lambda: 0.6620638370513916


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff3b69330d0>
Traceback (most recent call last):
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/multiprocessing/process.py", line 147, in join
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff3b69330d0>
Traceback (most recent call last):
    assert self._parent_pid == os.getpid(), 'can only join a child process'
  File "/nethome/bdevnani3/anaconda3/envs/p3/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
AssertionError: can only join a child process
    self._shutdown_workers()
  File "/nethome/b

In [53]:
# Evaluation code

# pass only test or valid loader here
def evaluate(data_loader, image_embeds, text_embeds, lam):
    model.eval()
    with torch.no_grad():
        top1, n = 0.0, 0.0
        for i, (images, target) in enumerate(tqdm(test_loader)):
            images = images.cuda()
            target = target.cuda()
            image_features = clip_model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            dot = lam*(image_features @ image_embeds.T) + (1-lam)*(image_features @ text_embeds.T)
            out = torch.argmax(dot, axis=1).cpu().numpy()
            top1 += torch.sum(torch.argmax(dot, axis=1) == target).item()
            n+=len(target)
    return top1/n

In [156]:
evaluate(test_loader,image_embeds,text_embeds,0.5)

  0%|          | 0/17 [00:00<?, ?it/s]

0.8192918192918193